In [ ]:
%pip install dotenv
%pip install datasets
%pip install scikit-learn
%pip install -r requirements.txt
%load_ext autoreload
%autoreload 2

In [4]:
import torch
import sys

sys.path.append(".")

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
print("CPU cores:", os.cpu_count())

CPU cores: 128


In [ ]:
from huggingface_hub import login
import os
from dotenv import load_dotenv
load_dotenv()

# Use your token to log in
login(token=os.getenv("hf_token"))

In [10]:
import sys
sys.path.append('/utils')

from utils.experiment_utils import BERT, BART, ROBERTA, POLITICS, run_experiment, DatasetConfig, ExperimentConfig
import json

In [ ]:
def run_custom(model, loss):
    # Model Selection (choose one)
    MODEL = f"{model}/epoch-1.pt"
    
    # Output directory
    OUTPUT_DIR = f"{model}/"
    
    # Dataset Configuration
    DATASET_CONFIG = DatasetConfig(
        custom_dataset="dragonslayer631/article-bias-prediction-media-splits-updated",
        theme=None,
        k_means=None,
        trunc=True,
        sentiment=False,
        no_undersampling=True,
        media_split=True
    )
    
    # Experiment Configuration
    EXPERIMENT_CONFIG = ExperimentConfig(
        loss_type=loss,  # Options: "standard", "focal", "weighted_asymmetric_focal"
        focal_alpha=1.0,
        focal_gamma=2.0,
        gamma_pos=1.0,
        gamma_neg=4.0,
        patience=3,
        save_model=False
    )
    

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # Run the experiment
    print("Starting experiment...")
    print(f"Model: {MODEL}")
    print(f"Loss type: {EXPERIMENT_CONFIG.loss_type}")
    print(f"Output directory: {OUTPUT_DIR}")
    print("-" * 50)
    
    try:
        results = run_experiment(
            model=MODEL,
            loc=OUTPUT_DIR,
            dataset_config=DATASET_CONFIG,
            experiment_config=EXPERIMENT_CONFIG
        )
        print("Experiment completed successfully!")
        print(f"Results saved to: {OUTPUT_DIR}")
        
        if isinstance(results, dict) and len(results) > 1:
            print(f"Multiple theme results: {list(results.keys())}")
        
    except Exception as e:
        print(f"Experiment failed with error: {str(e)}")
        raise e

In [ ]:
run_custom("triplet_mlm", "focal")

In [ ]:
run_custom("triplet_mlm", "weighted_asymmetric_focal")

In [ ]:
run_custom("triplet_mlm_themes", "focal")

In [ ]:
run_custom("triplet_mlm_themes", "weighted_asymmetric_focal")